## Dynamic Double Machine Learning (DML) on top of ClusterVAR for Prediction Purposes

This should serve as a toy example before we ship it off in an actual script and combine it with .py files to ship it out. I will also attempt to describe the math and logic to this, subjected to diagrams in which I will either draw my hand/tikz if time permits.

Lets go!

### $\Delta_1:$ ClusterVAR(p)/BlockVAR(p)

Let's recall the canonical VAR(p) model here and its assumptions. Let $y_t \in \mathbb{R}^N$ where $N$ is the number of features (or tickers) indexed by time $t$. For a VAR(p) model, we thus have

$$
y_t = \mu +  A_1 y_{t-1} + A_2 y_{t-2} + \cdots + A_p y_p + \varepsilon_t
$$


for coefficient matrices $A_i \in \mathbb{R}^{N \times N}$, mean vector $\mu \in \mathbb{R}^N$, with i.i.d. $\varepsilon_t \sim N(0,\Sigma_u)$ representing the innovation/error terms with covariance matrix $\Sigma_u$. Upon clustering, we have the same mathematical equation as in (1) above, with $N \rightarrow K$, where $K$ is the number of clusters.

In the perspective of DAGs (in preparation for causally-flavored prediction models that will follow), we can consider equation (1) as a **single stacked vector** for each $t$ with correlated (but i.i.d.) error with covariance matrix $\Sigma_u$. The corresponding DAGs will be given by:

- $y_{i} \rightarrow y_{j}$ for all $i < j$ (earlier nodes point forawrd in time to later nodes).
- $u_t \rightarrow y_t$ for all $t$ (there is only a "single" noise term pointing to each node in time, with the entire vector $y_t$ collapsed as a single node per time period $t$).

In statsmodels.tsa.api.VAR, this is done by stacking the lagged vectors up into

$$Y = XB + U$$
with 
$$\begin{aligned}
Y &= [y_t,...,y_{t-p}]' \in \mathbb{R}^{(T-p) \times K}, \text{ with each } y_i \in \mathbb{R}^K, \\
X &= ... \in \mathbb{R}^{(T-p) \times (K\cdot p + 1)}, \\
B &= ... \in \mathbb{R}, \\
U &= \text{Stacked Error Matrix}
\end{aligned}
$$

and running OLS to obtain

$$B = (X'X)^{-1}(X'Y).$$

### $\Delta_2:$ Lag Selection by Sensitivity Analysis

Here, we can select the lag by soaking up the confounding set to be as large as possible such that the coefficient of the "treatment effect edges" are relatively stable with respect to adding feature vectors with higher lag (with directed edges originating from an even earlier time). This is done via two different types of test. Assume that the treatment set only contains $y_{t-1}$ and we are considering adding $y_{t-2}$. To do so, we first

- Add $y_{t-2}$ in the treatment set and perform DML.
- Significance? (ie $\hat\theta_{-2} \neq 0$.) Keep it inside the treatment set.
- Drift? (ie $\hat\theta_{-1}$ has changed but without significance.) Move it to the confounding set. 
- Terminate if we are at a maximum lag window specificed or if both tests output "No".

Logic for Significance:
The newly added lagged vector is affecting the outcome directly.

Logic for Drift (without Significance):
The added lagged vector $y_{t-2}$ was omitted variable biasing $y_1$ but does not have its own casual effect. Hence, $y_{t-2} \rightarrow y_{t-1}$ but not $y_{t-2} \rightarrow y_t$. Hence, we obtain $y_{t-1} \leftarrow y_{t-2} \leftarrow W \rightarrow y_t$ (and of course, the "front-door path" $y_{t-1} \rightarrow y_t$). Conditioning also on $y_{t-2}$ or adding it into $W$ blocks the aforementioned backdoor path. (Here, $W$ is the confounding set.)

Even if there were no edges from $y_{t-2} to $y_t$, the overall "strength" from the confounding set will still make the edge from $U \rightarrow y_t$ valid and thus adjustable as a nuisance function.

Remark: c.f. Long Story Short: Omitted Variable Bias in Causal Machine Learning paper in 2022. We are basically doing this method but for time-series analysis.


### $\Delta_3$: A prediction model from DML and correlation analysis.

To be continued (I have the idea for this but I don't have the time to write this...)

In [ ]:
conda init powershell 
conda activate econml_env ## This uses Python Version 3.11